In [40]:
import pandas as pd
import os

In [41]:
# Available files
list_file_rte_definitif = [
	f"eCO2mix_RTE_Annuel-Definitif_{i}.xls" for i in range(2014,2021)
]
print(list_file_rte_definitif)

['eCO2mix_RTE_Annuel-Definitif_2014.xls', 'eCO2mix_RTE_Annuel-Definitif_2015.xls', 'eCO2mix_RTE_Annuel-Definitif_2016.xls', 'eCO2mix_RTE_Annuel-Definitif_2017.xls', 'eCO2mix_RTE_Annuel-Definitif_2018.xls', 'eCO2mix_RTE_Annuel-Definitif_2019.xls', 'eCO2mix_RTE_Annuel-Definitif_2020.xls']


In [42]:
# Traitement fichier définitif (2014 --> 2020 inclus)
df_definitif = pd.DataFrame()
for file in list_file_rte_definitif:
    df_year = pd.read_csv(
        os.path.join("raw_datasets",file),
        skipfooter=1,
        engine='python',
        index_col=False,
        skiprows=lambda x: x % 2 != 1 if x != 0 else False,
        encoding='latin-1',
        sep="\t"
        )
    df_definitif=pd.concat([df_definitif,df_year])
display(df_definitif.tail())

,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
17563,France,Données définitives,2020-12-31,21:30,64423,64400,63700,94,8,5712,...,5202,1839,715,492,291,337,ND,ND,ND,ND
17564,France,Données définitives,2020-12-31,22:00,63505,63800,63200,93,8,5583,...,5107,1543,496,494,287,335,ND,ND,ND,ND
17565,France,Données définitives,2020-12-31,22:30,64331,65800,65200,93,8,5573,...,4989,1292,855,492,293,337,ND,ND,ND,ND
17566,France,Données définitives,2020-12-31,23:00,66853,67500,67000,94,8,5731,...,5106,2251,791,496,291,337,ND,ND,ND,ND
17567,France,Données définitives,2020-12-31,23:30,66750,67600,67200,93,8,5690,...,5112,1477,616,499,288,332,ND,ND,ND,ND


In [43]:
# Traitement fichier consolide (2021 --> 31/05/2022)
empty_column = [
    ' Stockage batterie',
    'Déstockage batterie',
    'Eolien terrestre',
    'Eolien offshore'
]

df_consolide = pd.read_csv(
        "raw_datasets/eCO2mix_RTE_En-cours-Consolide.xls",
        skipfooter=0,
        index_col=False,
        engine='python',
        skiprows=lambda x: x % 2 != 1 if x != 0 else False,
        encoding='latin-1',
        sep="\t",
        usecols=lambda col: col not in empty_column
        )
df_consolide['Date'] = df_consolide['Date'].str.replace('/','-')
df_consolide["Date"] = pd.to_datetime(df_consolide['Date'], format='%d-%m-%Y').dt.date


# To avoid a bug on 15/04 at 23:30 where there is no empty line 
# between two dates
threshold_date = pd.to_datetime('2022-04-16')
df_consolide = df_consolide.loc[df_consolide["Date"]<threshold_date]

df_consolide_part2 = pd.read_csv(
        "raw_datasets/eCO2mix_RTE_En-cours-Consolide.xls",
        skipfooter=0,
        header = None,
        index_col=False,
        engine='python',
        skiprows=45120,
        encoding='latin-1',
        sep="\t",
        usecols=lambda column: column not in range(36, 40)
        )
df_consolide_part2 = df_consolide_part2.iloc[::2]
df_consolide_part2.columns=df_consolide.columns
df_consolide_part2['Date'] = df_consolide_part2['Date'].str.replace('/','-')
df_consolide_part2["Date"] = pd.to_datetime(df_consolide_part2['Date'], format='%d-%m-%Y').dt.date
df_consolide_final = pd.concat([df_consolide,df_consolide_part2])


/var/folders/t3/myv5sjjd0537bvgr90_h_fcc0000gn/T/ipykernel_71390/1904115515.py:26: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_consolide = df_consolide.loc[df_consolide["Date"]<threshold_date]


In [44]:
# Traitement fichier En cours temps réel (TR)
empty_column = [
    ' Stockage batterie',
    'Déstockage batterie',
    'Eolien terrestre',
    'Eolien offshore'
]
df_tr = pd.read_csv(
        "raw_datasets/eCO2mix_RTE_En-cours-TR.xls",
        index_col=False,
        engine='python',
        skiprows=lambda x: x % 2 != 0 if x != 0 else False,
        encoding='latin-1',
        sep="\t",
        usecols=lambda col: col not in empty_column
        )
# Suppression des lignes de prévisions (J+2)
# On supprime si plus de la moitié des colonnes sont vides, ce qui corresponds
# aux prédictions à J+2
nombre_colonnes = len(df_tr.columns)

# Définir le nombre minimum de valeurs non NaN requises pour conserver la ligne
thresh = nombre_colonnes // 2

# Supprimer les lignes où plus de la moitié des colonnes ont des valeurs NaN
df_tr = df_tr.dropna(thresh=len(df_tr.columns)//2)

In [45]:
# Rassemblement des fichiers
df_all = pd.concat([
	df_definitif,
	df_consolide_final,
	df_tr
])
# Ecriture dans fichier csv définitif
file_path = "clean_datasets/rte_production.csv"
try:
	df_all.to_csv(file_path, encoding='utf-8')
	print(f"Données écrites avec succès dans {file_path}")
except IOError as e:
	print(f"Erreur lors de l'écriture dans {file_path}: {e}")

Données écrites avec succès dans clean_datasets/rte_production.csv


In [46]:
display(df_all.head())

,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
0,France,Données définitives,2014-01-01,00:00,64660.0,63200,63200.0,454.0,0.0,2303.0,...,5662,2125,693.0,460,165.0,179.0,NaN,NaN,NaN,NaN
1,France,Données définitives,2014-01-01,00:30,63494.0,62100,61500.0,281.0,0.0,2367.0,...,5569,2450,495.0,592,170.0,181.0,NaN,NaN,NaN,NaN
2,France,Données définitives,2014-01-01,01:00,61362.0,59900,59900.0,281.0,0.0,2188.0,...,5457,2040,76.0,591,173.0,181.0,NaN,NaN,NaN,NaN
3,France,Données définitives,2014-01-01,01:30,61217.0,60600,60300.0,281.0,0.0,2190.0,...,5286,1849,0.0,595,174.0,180.0,NaN,NaN,NaN,NaN
4,France,Données définitives,2014-01-01,02:00,60748.0,59900,60200.0,281.0,0.0,2187.0,...,5201,1592,0.0,597,174.0,180.0,NaN,NaN,NaN,NaN
